# Prescriptive Order Management (Starter Notebook)

This notebook reads orders from `orders.csv`, cleans data, and optimizes a shipping schedule using linear programming (PuLP).

In [ ]:
!pip install pandas
!pip install matplotlib

import pandas as pd
import numpy as np
from datetime import date, timedelta
import matplotlib.pyplot as plt

# Parameters
CSV_PATH = "orders.csv"
TODAY = pd.to_datetime("2025-09-19")  # change to pd.to_datetime(date.today()) for real-time
HORIZON_DAYS = 7
DEFAULT_DAILY_CAPACITY = 80

# Read CSV
orders = pd.read_csv(CSV_PATH, parse_dates=["due_date"])
orders.head()


In [ ]:

# Build planning horizon & capacity (uniform by default)
horizon = pd.date_range(TODAY, periods=HORIZON_DAYS, freq="D")
capacity = pd.Series([DEFAULT_DAILY_CAPACITY] * HORIZON_DAYS, index=horizon)
capacity


In [ ]:

# Data validation & derived fields
assert orders["qty"].ge(0).all(), "Quantities must be non-negative"
assert orders["priority"].between(1,3).all(), "Priority must be 1..3"

orders["penalty_per_day"] = orders["penalty_per_day"].fillna(0.0)
orders["est_ship_days"] = orders["est_ship_days"].fillna(1).astype(int)

orders["latest_ship_date"] = orders["due_date"] - pd.to_timedelta(orders["est_ship_days"], unit="D")
orders["latest_ship_date"] = orders["latest_ship_date"].clip(lower=TODAY)

# Keep orders whose latest_ship_date is inside planning horizon
orders = orders[orders["latest_ship_date"] <= horizon.max()].copy()

mask_ok = (
    orders["qty"].gt(0) &
    orders["due_date"].ge(TODAY) &
    orders["est_ship_days"].between(1, 10)
)
issues = orders.loc[~mask_ok].copy()
orders = orders.loc[mask_ok].copy()
print("Removed problematic rows:")
display(issues if not issues.empty else "None")
orders


In [ ]:

# Quick EDA
print("Orders:", len(orders))
print("Total qty:", orders["qty"].sum())
print(orders.groupby("sku")["qty"].sum().sort_values(ascending=False))

orders["slack_days"] = (orders["latest_ship_date"] - TODAY).dt.days
orders[["order_id","priority","due_date","latest_ship_date","slack_days","qty"]]    .sort_values(["slack_days","priority"]).head(10)


In [ ]:

# Plot quantity by due date
orders_by_due = orders.groupby(orders["due_date"].dt.date)["qty"].sum()
plt.figure()
orders_by_due.plot(kind="bar")
plt.title("Qty by Due Date")
plt.xlabel("Due Date")
plt.ylabel("Total Qty")
plt.tight_layout()


## Optimization with PuLP

In [ ]:
!pip install pulp

import pulp as pl

# Index sets
O = orders.index.tolist()
D = list(range(len(horizon)))  # day indices

# Helper maps
qty = orders["qty"].to_dict()
latest_ship_date = orders["latest_ship_date"].dt.date.to_dict()
penalty = orders["penalty_per_day"].to_dict()
order_ids = orders["order_id"].to_dict()

def ship_cost(day_idx, o):
    weekday = horizon[day_idx].weekday()  # 0=Mon
    base = 1.0
    if weekday >= 5:  # weekend premium
        base += 0.5
    return base

def late_penalty(day_idx, o):
    ship_day = horizon[day_idx].date()
    latest = latest_ship_date[o]
    days_late = max((ship_day - latest).days, 0)
    return days_late * penalty[o]

# Model
m = pl.LpProblem("OrderShippingSchedule", pl.LpMinimize)

# Decision variables
x = pl.LpVariable.dicts("x", ((o, d) for o in O for d in D), lowBound=0, upBound=1, cat=pl.LpBinary)

# Objective
m += pl.lpSum((ship_cost(d, o) + late_penalty(d, o)) * x[(o, d)] for o in O for d in D)

# Constraints
for o in O:
    m += pl.lpSum(x[(o, d)] for d in D) <= 1
for d in D:
    m += pl.lpSum(qty[o] * x[(o, d)] for o in O) <= capacity.iloc[d]

_ = m.solve(pl.PULP_CBC_CMD(msg=False))
print("Status:", pl.LpStatus[m.status])


In [ ]:
# Extract recommendations
assignments = []
total_assigned = 0

for o in O:
    for d in D:
        if pl.value(x[(o,d)]) and pl.value(x[(o,d)]) > 0.5:
            assignments.append({
                "order_id": order_ids[o],
                "ship_date": horizon[d].date(),
                "qty": qty[o],
                "cost_component": ship_cost(d, o),
                "late_penalty": late_penalty(d, o),
            })
            total_assigned += 1

print(f"Assigned {total_assigned} orders out of {len(O)} total orders")

if not assignments:
    print("No orders were assigned! Check constraints and capacity.")
    assign_df = pd.DataFrame(columns=["order_id", "ship_date", "qty", "cost_component", "late_penalty"])
else:
    assign_df = pd.DataFrame(assignments)

result = orders[["order_id","sku","qty","due_date","latest_ship_date","priority","penalty_per_day"]] \
    .merge(assign_df, on="order_id", how="left")

result["days_late"] = (pd.to_datetime(result["ship_date"]) - result["latest_ship_date"]).dt.days.clip(lower=0)
result["total_cost"] = result["cost_component"].fillna(0) + result["late_penalty"].fillna(0)

print("Total cost:", result["total_cost"].sum())
print("Late orders:", (result["days_late"]>0).sum())
print("Unassigned orders:", result["ship_date"].isna().sum())
result.sort_values(["priority","days_late","total_cost"]).head(20)

In [ ]:
# Capacity vs plan plot
assigned_orders = result.dropna(subset=["ship_date"])
print(f"Orders with ship dates: {len(assigned_orders)}")

if len(assigned_orders) > 0:
    plan = assigned_orders.groupby("ship_date")["qty"].sum()
    print(f"Plan dates: {plan.index.tolist()}")
else:
    # Create empty series with same date index as capacity for plotting
    plan = pd.Series([], dtype=float, name="qty")
    plan.index = pd.to_datetime([])
    print("No orders assigned to dates - showing empty plan")

cap_plot = capacity.copy()
cap_plot.index = cap_plot.index.date

plt.figure(figsize=(10, 6))
cap_plot.plot(label="Capacity", marker='o')

if len(plan) > 0:
    plan.plot(kind="bar", label="Planned Qty", alpha=0.7)
else:
    plt.bar([], [], label="Planned Qty")  # Empty bar plot for legend

plt.legend()
plt.title("Capacity vs. Planned Shipments")
plt.xlabel("Date")
plt.ylabel("Units")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### What-if ideas
- Change `DEFAULT_DAILY_CAPACITY` or edit `capacity` for specific dates.
- Increase `penalty_per_day` for high-priority orders.
- Extend `HORIZON_DAYS`.
- Add priority-reservation constraints.
